### SCD TYPE 2
Effective start , End date & Inuse flag is present in SCD TYPE 2.
RECORDS ARE NOT OVERWRITTEN.
OLD RECORD IS UPDATED WITH Inuse flad No & New End date.
New record with Inuse flag Yes is inserted

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.default.scdtype2_source
(
  Product_id int primary key,
  product_name STRING,
  Product_cat STRING,
  ProcessDate Date
)

In [0]:
%sql
insert into datamodeling.default.scdtype2_source
values
(1,'Product1','Category1',current_date()),
(2,'Product2','Category2',current_date()),
(3,'Product3','Category3',current_date());

select * from datamodeling.default.scdtype2_source;



In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.scdtype2_source
(
  Product_id int primary key,
  product_name STRING,
  Product_cat STRING,
  ProcessDate Date,
  startDate DATE, 
  EndDate Date,
  InUse STRING
)

In [0]:
%sql
insert into datamodeling.gold.scdtype2_source
values
(1,'Product1','Category1',current_date(),current_date()-5,'3000-10-10','Y'),
(2,'Product2','Category2',current_date(),current_date()-5,'3000-10-10','Y'),
(3,'Product','Category3',current_date(),current_date()-5,'3000-10-10','Y');   

In [0]:
%sql
Select * from datamodeling.gold.scdtype2_source

In [0]:
df = spark.sql(
    """
    SELECT *,
        current_timestamp AS startDate,
        CAST('3000-01-01' AS timestamp) AS EndDate,
        'Y' AS InUse
    FROM datamodeling.default.scdtype2_source
    """
)
df.createOrReplaceTempView("scdtype2_source")

In [0]:
%sql
Select * from scdtype2_source

### DEACTIVATE EXISTING RECORD

In [0]:
%sql
MERGE INTO datamodeling.gold.scdtype2_source target
USING scdtype2_source
ON scdtype2_source.PRODUCT_ID = target.PRODUCT_ID AND 
target.InUse = 'Y'
WHEN MATCHED AND (
  scdtype2_source.PRODUCT_NAME <> target.PRODUCT_NAME OR
  scdtype2_source.PRODUCT_CAT <> target.PRODUCT_CAT OR
  scdtype2_source.PROCESSDATE <> target.PROCESSDATE
) THEN UPDATE SET target.EndDate = current_timestamp(),
                  target.InUse = 'N'

In [0]:
%sql
MERGE INTO datamodeling.gold.scdtype2_source target
USING scdtype2_source
ON scdtype2_source.PRODUCT_ID = target.PRODUCT_ID AND 
target.InUse = 'Y'
WHEN NOT MATCHED THEN INSERT *

In [0]:
%sql
Select * from datamodeling.GOLD.scdtype2_source

Lets update & insert a row in source table to test merge queries

In [0]:
%sql
Update datamodeling.default.scdtype2_source
set product_cat='New_Category' where product_id = 2;

Insert into datamodeling.default.scdtype2_source
values
(4,'Product4','Category4',current_date);

In [0]:
%sql
-- Select * from datamodeling.default.scdtype2_source

Drop table datamodeling.gold.scdtype2_source